# Wess Zumino 

## N=1 in 1+1

The Hamiltonian with a central difference for the derivative is given by 

\begin{align}
    H=\sum_n&\left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2
            +aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
            &\left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right)
            +\frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right],
\end{align}

In [1]:
import sympy as sp
from sympy_utils import *

#pn, qn, qnP1, qnM1 = sp.symbols('p_n, q_n, q_{n+1}, q_{n-1}', commutative=False)
#xn, xdn, xnP1, xdnP1 = sp.symbols('\chi_n', '\chi^{\dagger}_n', '\chi_{n+1}', '\chi^{\dagger}_{n+1}')
n=sp.symbols('n')
aLat=sp.symbols('a')
V=sp.Function('V')
p=sp.IndexedBase('p',commutative=False)
q=sp.IndexedBase('q',commutative=False)
x=sp.IndexedBase('\chi',commutative=False)
xd=sp.IndexedBase('\chi^{\dagger}',commutative=False)

In [2]:
bosonNI=sp.expand( p[n]**2/(2*aLat) + ((aLat/2)*(q[n+1]-q[n-1])/(2*aLat))**2 )
bosonI=sp.expand( (aLat/2)*V(q[n])**2 + aLat*V(q[n])*(q[n+1]-q[n-1])/(2*aLat) )
fermionNI=sp.expand( (xd[n]*x[n+1]+xd[n+1]*x[n])/(2*aLat) )
fermionI=sp.expand( (-1)**n*sp.diff(V(q[n]),q[n])*(xd[n]*x[n]-(1/2)) )

totHam=bosonNI+bosonI+fermionNI+fermionI

In [3]:
bosonNI+bosonI

a*V(q[n])**2/2 + V(q[n])*q[n + 1]/2 - V(q[n])*q[n - 1]/2 + q[n + 1]**2/16 - q[n + 1]*q[n - 1]/8 + q[n - 1]**2/16 + p[n]**2/(2*a)

In [4]:
fermionNI + fermionI

(-1)**n*Derivative(V(q[n]), q[n])*\chi[n]*\chi^{\dagger}[n] - 0.5*(-1)**n*Derivative(V(q[n]), q[n]) + \chi[n + 1]*\chi^{\dagger}[n]/(2*a) + \chi[n]*\chi^{\dagger}[n + 1]/(2*a)

In [5]:
N=3
aVal=1

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {q[-1]: q[N-1], q[N]: q[0],
                          x[N]: x[0], xd[N]: xd[0],
                         }
elif bcType == 'dirichlet':
    boundaryConditions = {q[-1]: 0, q[N]: 0,
                          x[N]: 0, xd[N]: 0
                         }

In [6]:
ham=0
totHam=bosonNI+bosonI+fermionNI+fermionI
for i in range(0,N):
    ham+=totHam.subs(n,i).subs(boundaryConditions)
# ham.subs(boundaryConditions).doit() # this doesn't work?

def potential(n):
    return 0

for n in range(0,N):
    ham=ham.subs(V(q[n]),potential(n))
ham=ham.subs(aLat,aVal).simplify().expand()
ham

\chi[0]*\chi^{\dagger}[1]/2 + \chi[0]*\chi^{\dagger}[2]/2 + \chi[1]*\chi^{\dagger}[0]/2 + \chi[1]*\chi^{\dagger}[2]/2 + \chi[2]*\chi^{\dagger}[0]/2 + \chi[2]*\chi^{\dagger}[1]/2 + p[0]**2/2 + p[1]**2/2 + p[2]**2/2 + q[0]**2/8 - q[0]*q[1]/8 - q[0]*q[2]/8 + q[1]**2/8 - q[1]*q[2]/8 + q[2]**2/8

In [7]:
#ham=0
#totHam=bosonNI+bosonI+fermionNI+fermionI
#for i in range(0,N):
#    ham+=totHam.subs(n,i).subs(boundaryConditions)
# ham.subs(boundaryConditions).doit() # this doesn't work?

#def potential(n):
#    return q[n]

#for n in range(0,N):
#    ham=ham.subs(V(q[n]),potential(n))
#ham=ham.subs(aLat,aVal).simplify().expand()
#ham

In [8]:
a = sp.IndexedBase('a',commutative=False)
adag = sp.IndexedBase('a^{\dagger}',commutative=False)

m=1

HOdofSubs = {}
for i in range(0,N):
    HOdofSubs[q[i]] = 0.5*sp.sqrt(2/m)*(a[i] + adag[i])
    HOdofSubs[p[i]] = complex(0,1)*sp.sqrt(2*m)*(adag[i] - a[i])/2
    
hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam

\chi[0]*\chi^{\dagger}[1]/2 + \chi[0]*\chi^{\dagger}[2]/2 + \chi[1]*\chi^{\dagger}[0]/2 + \chi[1]*\chi^{\dagger}[2]/2 + \chi[2]*\chi^{\dagger}[0]/2 + \chi[2]*\chi^{\dagger}[1]/2 - 0.1875*a[0]**2 - 0.0625*a[0]*a[1] - 0.0625*a[0]*a[2] + 0.625*a[0]*a^{\dagger}[0] - 0.0625*a[0]*a^{\dagger}[1] - 0.0625*a[0]*a^{\dagger}[2] - 0.1875*a[1]**2 - 0.0625*a[1]*a[2] - 0.0625*a[1]*a^{\dagger}[0] + 0.625*a[1]*a^{\dagger}[1] - 0.0625*a[1]*a^{\dagger}[2] - 0.1875*a[2]**2 - 0.0625*a[2]*a^{\dagger}[0] - 0.0625*a[2]*a^{\dagger}[1] + 0.625*a[2]*a^{\dagger}[2] - 0.1875*a^{\dagger}[0]**2 - 0.0625*a^{\dagger}[0]*a^{\dagger}[1] - 0.0625*a^{\dagger}[0]*a^{\dagger}[2] - 0.1875*a^{\dagger}[1]**2 - 0.0625*a^{\dagger}[1]*a^{\dagger}[2] - 0.1875*a^{\dagger}[2]**2

In [19]:
# should probably error handle better so that when this breaks
# I know what was wrong with my expression

cutoff=2



In [23]:
hamMat=convert_to_matrix(hoHam,cutoff,N)

(64, 64)


In [29]:
hamMat

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0.625000000000000, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0.625000000000000, 0, 0]], dtype=object)

In [34]:
type(hamMat[2,0])

sympy.core.numbers.Float

In [35]:
np.linalg.eig(hamMat.astype(np.complex64))

(array([ 2.5568968e-01-5.61964154e-01j,  2.5568968e-01+5.61964154e-01j,
        -7.6090592e-01+4.59368255e-16j, -1.2609059e+00-3.88260448e-16j,
         1.2556897e+00-5.61964154e-01j,  1.2556897e+00+5.61964154e-01j,
         1.2556897e+00-5.61964154e-01j,  1.2556897e+00+5.61964154e-01j,
         9.9952650e-01+2.67553356e-16j, -4.7347872e-04-6.82905954e-17j,
         2.3909411e-01-2.12873726e-16j,  9.9952650e-01-2.18729808e-16j,
         2.3909411e-01+5.81994287e-17j,  2.5568968e-01-5.61964154e-01j,
         2.5568968e-01+5.61964154e-01j, -7.6090592e-01+2.22017593e-16j,
        -1.2609059e+00+5.27355937e-16j, -1.2609059e+00+2.41560244e-16j,
        -1.2609059e+00-1.42247325e-16j, -4.7347872e-04-1.06610412e-16j,
         1.0625000e+00-1.34702582e-10j,  1.0625000e+00-1.02661319e-08j,
         1.0625000e+00-8.16254531e-09j,  1.0625000e+00-8.37479774e-09j,
         1.0625000e+00+1.34701833e-10j,  1.0625000e+00+8.16254619e-09j,
         1.0625000e+00+1.02661337e-08j,  1.0625000e+00+8.3747968